# Load API

Loading API is central to a lot of nilmtk operations and provides a great deal of flexibility. We'll now look at ways in which we can load data from the nilmtk DataStore to the memory based on different conditional queries. To see the full range of possible queries, we'll use the [iAWE data set](http://iawe.github.io) (whose HDF5 file can be downloaded [here](https://copy.com/C2sIt1UfDx1mfPlC)) alongwith the REDD data set. 

The `load` function returns a generator of DataFrames loaded from the DataStore based on the conditions specified. If no conditions are specified, then all data from all the columns is loaded.

In [1]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
rcParams['figure.figsize'] = (13, 6)
plt.style.use('ggplot')
from nilmtk import DataSet


iawe = DataSet('/data/iawe/iawe.h5')
elec = iawe.buildings[1].elec
elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=3, building=1, dataset='iAWE', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=4, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=1)])
  ElecMeter(instance=5, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=2)])
  ElecMeter(instance=6, building=1, dataset='iAWE', appliances=[Appliance(type='washing machine', instance=1)])
  ElecMeter(instance=7, building=1, dataset='iAWE', appliances=[Appliance(type='computer', instance=1)])
  ElecMeter(instance=8, building=1, dataset='iAWE', appliances=[Appliance(type='clothes iron', instance=1)])
  ElecMeter(instance=9, building=1, dataset='iAWE', appliances=[Appliance(type='unknown', instance=1)])
  ElecMeter(instance=10, building=1, dataset='iAWE', appliances=[A

Let us see what all measurements do we have for fridge.

In [2]:
fridge = elec['fridge']

In [3]:
fridge.available_columns()

[('current', None),
 ('power', 'apparent'),
 ('frequency', None),
 ('voltage', None),
 ('power', 'active'),
 ('power factor', None),
 ('power', 'reactive')]

## Loading data

### Loading all power columns (default)

In [4]:
df = fridge.load().next()

{'cols': [('current', None), ('power', 'apparent'), ('frequency', None), ('voltage', None), ('power', 'active'), ('power factor', None), ('power', 'reactive')]}


In [5]:
df.head()

physical_quantity            power                
type                      apparent active reactive
2013-06-07 05:30:00+05:30    2.486  0.111    2.483
2013-06-07 05:30:01+05:30    2.555  0.200    2.547
2013-06-07 05:30:02+05:30    2.485  0.152    2.480
2013-06-07 05:30:03+05:30    2.449  0.159    2.444
2013-06-07 05:30:04+05:30    2.519  0.215    2.510

### Loading by specifying column names

In [6]:
df = fridge.load(cols = [('power', 'active')]).next()
df.head()

{'cols': [('power', 'active')]}


physical_quantity,power
type,active
2013-06-07 05:30:00+05:30,0.111
2013-06-07 05:30:01+05:30,0.200
2013-06-07 05:30:02+05:30,0.152
2013-06-07 05:30:03+05:30,0.159
2013-06-07 05:30:04+05:30,0.215


In [11]:
df = fridge.load(cols = [('voltage', None)]).next()
df.head()

{'cols': [('voltage', None)]}


2013-06-07 05:30:00+05:30
2013-06-07 05:30:01+05:30
2013-06-07 05:30:02+05:30
2013-06-07 05:30:03+05:30
2013-06-07 05:30:04+05:30


### Loading by specifying physical_type

In [8]:
df = fridge.load(physical_quantity = 'power').next()
df.head()

{'cols': [('power', 'apparent'), ('power', 'active'), ('power', 'reactive')]}


physical_quantity            power                
type                      apparent active reactive
2013-06-07 05:30:00+05:30    2.486  0.111    2.483
2013-06-07 05:30:01+05:30    2.555  0.200    2.547
2013-06-07 05:30:02+05:30    2.485  0.152    2.480
2013-06-07 05:30:03+05:30    2.449  0.159    2.444
2013-06-07 05:30:04+05:30    2.519  0.215    2.510

### Loading by specifying AC type

In [9]:
df = fridge.load(ac_type = 'active').next()
df.head()

{'cols': [('power', 'active')]}


physical_quantity,power
type,active
2013-06-07 05:30:00+05:30,0.111
2013-06-07 05:30:01+05:30,0.200
2013-06-07 05:30:02+05:30,0.152
2013-06-07 05:30:03+05:30,0.159
2013-06-07 05:30:04+05:30,0.215


### Loading by resmapling to a specified period

In [10]:
df = fridge.load(ac_type = 'active', sample_period=60).next()
df.head()

{'cols': [('power', 'active')]}


physical_quantity,power
type,active
2013-06-07 05:30:00+05:30,0.157583
2013-06-07 05:31:00+05:30,0.160567
2013-06-07 05:32:00+05:30,0.158170
2013-06-07 05:33:00+05:30,105.332802
2013-06-07 05:34:00+05:30,120.265068
